# Building Generative AI Applications with Gradio

## import 

In [1]:
import os
import io
import IPython.display
from PIL import Image
import base64
from transformers import pipeline
import gradio as gr

import warnings
warnings.filterwarnings("ignore") #, message=".*`max_length` will default to 20.*")

# Se você também quiser ignorar outros avisos, pode adicionar mais linhas como esta:
# warnings.filterwarnings("ignore", message=".*mensagem de aviso a ser ignorada.*")

# Image Captioning app

In [2]:
task = "image-to-text"
model = "Salesforce/blip-image-captioning-base"
image_captioner = pipeline("image-to-text") #, model = "Salesforce/blip-image-captioning-base")

No model was supplied, defaulted to ydshieh/vit-gpt2-coco-en and revision 65636df (https://huggingface.co/ydshieh/vit-gpt2-coco-en).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [33]:
img_url = "chopper.webp"
#img_url = "luffy.webp"
display(IPython.display.Image(url=img_url))
image_captioner(img_url)[0]['generated_text']

'a painting of a bear with a cartoon character on it '

In [4]:
def captioner(image):
    result = image_captioner(image)
    return result[0]['generated_text']

gr.close_all()
demo = gr.Interface(fn=captioner,
                    inputs=[gr.Image(label="Upload image", type="pil")],
                    outputs=[gr.Textbox(label="Caption")],
                    title="Image Captioning with BLIP",
                    description="Caption any image using the BLIP model",
                    allow_flagging="never",
                    #examples=["christmas_dog.jpeg", "bird_flight.jpeg", "cow.jpeg"]
                   )

demo.launch() #share=True) #, server_port=int(os.environ['PORT1']))

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


# Text Summarization app

In [5]:
from transformers import pipeline

get_completion = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']

In [6]:
text = ('''The tower is 324 metres (1,063 ft) tall, about the same height
        as an 81-storey building, and the tallest structure in Paris. 
        Its base is square, measuring 125 metres (410 ft) on each side. 
        During its construction, the Eiffel Tower surpassed the Washington 
        Monument to become the tallest man-made structure in the world,
        a title it held for 41 years until the Chrysler Building
        in New York City was finished in 1930. It was the first structure 
        to reach a height of 300 metres. Due to the addition of a broadcasting 
        aerial at the top of the tower in 1957, it is now taller than the 
        Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the 
        Eiffel Tower is the second tallest free-standing structure in France 
        after the Millau Viaduct.''')

summarize(text)

' The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building . It is the tallest structure in Paris and the second tallest free-standing structure in France after the Millau Viaduct . It was the first structure in the world to reach a height of 300 metres .'

In [7]:
import gradio as gr

# def summarize(input):
#     output = get_completion(input)
#     return output[0]['summary_text']

gr.close_all()
demo = gr.Interface(fn=summarize, 
                    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=3)],
                    title="Text summarization with distilbart-cnn",
                    description="Summarize any text using the `shleifer/distilbart-cnn-12-6` model under the hood!"
                   )
demo.launch() #share=True, server_port=int(os.environ['PORT2']))

Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [8]:
gr.close_all()

Closing server running on port: 7860
Closing server running on port: 7860


# Named Entity Recognition app

In [9]:
from transformers import pipeline

get_completion = pipeline("ner", model="dslim/bert-base-NER")

def ner(input):
    output = get_completion(input)
    return {"text": input, "entities": output}

In [10]:
ner("My name is Poli and work at HuggingFace")

{'text': 'My name is Poli and work at HuggingFace',
 'entities': [{'entity': 'B-PER',
   'score': 0.9872257,
   'index': 4,
   'word': 'Pol',
   'start': 11,
   'end': 14},
  {'entity': 'B-PER',
   'score': 0.44197148,
   'index': 5,
   'word': '##i',
   'start': 14,
   'end': 15},
  {'entity': 'B-ORG',
   'score': 0.9405219,
   'index': 9,
   'word': 'Hu',
   'start': 28,
   'end': 30},
  {'entity': 'I-ORG',
   'score': 0.8267373,
   'index': 10,
   'word': '##gging',
   'start': 30,
   'end': 35},
  {'entity': 'I-ORG',
   'score': 0.9957853,
   'index': 11,
   'word': '##F',
   'start': 35,
   'end': 36},
  {'entity': 'I-ORG',
   'score': 0.9153054,
   'index': 12,
   'word': '##ace',
   'start': 36,
   'end': 39}]}

In [11]:
gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    #Here we introduce a new tag, examples, easy to use examples for your application
                    examples=["My name is Andrew and I live in California", "My name is Poli and work at HuggingFace"])
demo.launch() #share=True, server_port=int(os.environ['PORT3']))

Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [12]:
gr.close_all()

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860


In [13]:
def merge_tokens(tokens):
    '''
    WHAT: Faz um loop entre os tokens para concatenar os tokens 
    com entidades I-* (intermediate token) aos B-* (begining token). 
    
    '''
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['entity'].startswith('I-') and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
            # Se a lista merged_tokens não estiver vazia.
            # o token atual for um token intermediário (começa com 'I-').
            # a entidade do último token processado terminar com a mesma entidade do token atual, excluindo o prefixo 'I-'.
            
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:            
            merged_tokens.append(token)

    return merged_tokens

def ner(input):
    """
    WHAT: Aplica a task de NER com o hugginface pipeline e concatena os tokens com a mesma entidade.
    RETURN: retorna um dicionário com o token e suas entidades.
    """
    output = get_completion(input) 
    merged_tokens = merge_tokens(output)
    return {"text": input, "entities": merged_tokens}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    examples=["My name is Andrew, I'm building DeeplearningAI and I live in California", "My name is Poli, I live in Vienna and work at HuggingFace"])

demo.launch() #share=True, server_port=int(os.environ['PORT4']))

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [14]:
gr.close_all()

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860


# Image Generation App

In [25]:
# from diffusers import DiffusionPipeline

# pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")

# def get_completion(prompt):
#     return pipeline(prompt).images[0]    

In [16]:
# prompt = "a dog in a park"

# result = get_completion(prompt)
# IPython.display.HTML(f'<img src="data:image/png;base64,{result}" />')

In [17]:
# import gradio as gr 

# #A helper function to convert the PIL image to base64
# #so you can send it to the API
# def base64_to_pil(img_base64):
#     base64_decoded = base64.b64decode(img_base64)
#     byte_stream = io.BytesIO(base64_decoded)
#     pil_image = Image.open(byte_stream)
#     return pil_image

# def generate(prompt):
#     output = get_completion(prompt)
#     result_image = base64_to_pil(output)
#     return result_image

# gr.close_all()
# demo = gr.Interface(fn=generate,
#                     inputs=[gr.Textbox(label="Your prompt")],
#                     outputs=[gr.Image(label="Result")],
#                     title="Image Generation with Stable Diffusion",
#                     description="Generate any image with Stable Diffusion",
#                     allow_flagging="never",
#                     examples=["the spirit of a tamagotchi wandering in the city of Vienna","a mecha robot in a favela"])

# demo.launch(share=True, server_port=int(os.environ['PORT1']))

In [18]:
# demo.close()

In [19]:
# import gradio as gr 

# #A helper function to convert the PIL image to base64 
# # so you can send it to the API
# def base64_to_pil(img_base64):
#     base64_decoded = base64.b64decode(img_base64)
#     byte_stream = io.BytesIO(base64_decoded)
#     pil_image = Image.open(byte_stream)
#     return pil_image

# def generate(prompt, negative_prompt, steps, guidance, width, height):
#     params = {
#         "negative_prompt": negative_prompt,
#         "num_inference_steps": steps,
#         "guidance_scale": guidance,
#         "width": width,
#         "height": height
#     }
    
#     output = get_completion(prompt, params)
#     pil_image = base64_to_pil(output)
#     return pil_image

# gr.close_all()
# demo = gr.Interface(fn=generate,
#                     inputs=[
#                         gr.Textbox(label="Your prompt"),
#                         gr.Textbox(label="Negative prompt"),
#                         gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
#                                  info="In how many steps will the denoiser denoise the image?"),
#                         gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7, 
#                                   info="Controls how much the text prompt influences the result"),
#                         gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512),
#                         gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512),
#                     ],
#                     outputs=[gr.Image(label="Result")],
#                     title="Image Generation with Stable Diffusion",
#                     description="Generate any image with Stable Diffusion",
#                     allow_flagging="never"
#                     )

# demo.launch() #share=True, server_port=int(os.environ['PORT2']))

In [20]:
# demo.close()

In [21]:
# with gr.Blocks() as demo:
#     gr.Markdown("# Image Generation with Stable Diffusion")
#     prompt = gr.Textbox(label="Your prompt")
#     with gr.Row():
#         with gr.Column():
#             negative_prompt = gr.Textbox(label="Negative prompt")
#             steps = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
#                       info="In many steps will the denoiser denoise the image?")
#             guidance = gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7,
#                       info="Controls how much the text prompt influences the result")
#             width = gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512)
#             height = gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512)
#             btn = gr.Button("Submit")
#         with gr.Column():
#             output = gr.Image(label="Result")

#     btn.click(fn=generate, inputs=[prompt,negative_prompt,steps,guidance,width,height], outputs=[output])
# gr.close_all()
# demo.launch() #share=True, server_port=int(os.environ['PORT3']))

In [22]:
# with gr.Blocks() as demo:
#     gr.Markdown("# Image Generation with Stable Diffusion")
#     with gr.Row():
#         with gr.Column(scale=4):
#             prompt = gr.Textbox(label="Your prompt") #Give prompt some real estate
#         with gr.Column(scale=1, min_width=50):
#             btn = gr.Button("Submit") #Submit button side by side!
#     with gr.Accordion("Advanced options", open=False): #Let's hide the advanced options!
#             negative_prompt = gr.Textbox(label="Negative prompt")
#             with gr.Row():
#                 with gr.Column():
#                     steps = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
#                       info="In many steps will the denoiser denoise the image?")
#                     guidance = gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7,
#                       info="Controls how much the text prompt influences the result")
#                 with gr.Column():
#                     width = gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512)
#                     height = gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512)
#     output = gr.Image(label="Result") #Move the output up too
            
#     btn.click(fn=generate, inputs=[prompt,negative_prompt,steps,guidance,width,height], outputs=[output])

# gr.close_all()
# demo.launch() #share=True, server_port=int(os.environ['PORT4']))

In [23]:
# gr.close_all()

# Describe-and-Generate game

In [ ]:
"""
to-do:
coletar o output do image captioner
colocar o output do image captioner como input do image generation app

"""

In [ ]:
# #Bringing the functions from lessons 3 and 4!
# def image_to_base64_str(pil_image):
#     byte_arr = io.BytesIO()
#     pil_image.save(byte_arr, format='PNG')
#     byte_arr = byte_arr.getvalue()
#     return str(base64.b64encode(byte_arr).decode('utf-8'))

# def base64_to_pil(img_base64):
#     base64_decoded = base64.b64decode(img_base64)
#     byte_stream = io.BytesIO(base64_decoded)
#     pil_image = Image.open(byte_stream)
#     return pil_image

# def captioner(image):
#     base64_image = image_to_base64_str(image)
#     result = get_completion(base64_image, None, ITT_ENDPOINT)
#     return result[0]['generated_text']

# def generate(prompt):
#     output = get_completion(prompt, None, TTI_ENDPOINT)
#     result_image = base64_to_pil(output)
#     return result_image

In [ ]:
# import gradio as gr 
# with gr.Blocks() as demo:
#     gr.Markdown("# Describe-and-Generate game 🖍️")
#     image_upload = gr.Image(label="Your first image",type="pil")
#     btn_caption = gr.Button("Generate caption")
#     caption = gr.Textbox(label="Generated caption")
    
#     btn_caption.click(fn=captioner, inputs=[image_upload], outputs=[caption])

# gr.close_all()
# demo.launch(share=True, server_port=int(os.environ['PORT1']))

In [ ]:
# with gr.Blocks() as demo:
#     gr.Markdown("# Describe-and-Generate game 🖍️")
#     image_upload = gr.Image(label="Your first image",type="pil")
#     btn_caption = gr.Button("Generate caption")
#     caption = gr.Textbox(label="Generated caption")
#     btn_image = gr.Button("Generate image")
#     image_output = gr.Image(label="Generated Image")
#     btn_caption.click(fn=captioner, inputs=[image_upload], outputs=[caption])
#     btn_image.click(fn=generate, inputs=[caption], outputs=[image_output])

# gr.close_all()
# demo.launch(share=True, server_port=int(os.environ['PORT2']))

In [ ]:
# def caption_and_generate(image):
#     caption = captioner(image)
#     image = generate(caption)
#     return [caption, image]

# with gr.Blocks() as demo:
#     gr.Markdown("# Describe-and-Generate game 🖍️")
#     image_upload = gr.Image(label="Your first image",type="pil")
#     btn_all = gr.Button("Caption and generate")
#     caption = gr.Textbox(label="Generated caption")
#     image_output = gr.Image(label="Generated Image")

#     btn_all.click(fn=caption_and_generate, inputs=[image_upload], outputs=[caption, image_output])

# gr.close_all()
# demo.launch(share=True, server_port=int(os.environ['PORT3']))

In [ ]:
# gr.close_all()

# Chat App

In [ ]:
from transformers import pipeline

task = "text-generation"
model = "tiiuae/falcon-7b-instruct" # or "tiiuae/falcon-40b-instruct"
get_completion = pipeline(task=task, model=model, trust_remote_code=True)

# def summarize(input):
#     output = get_completion(input)
#     return output[0]['summary_text']

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [ ]:
https://huggingface.co/spaces/mikeee/falcon-7b-ggml/tree/main

In [ ]:
#### exemplo

# from transformers import pipeline

# # Carregar o modelo pré-treinado
# generator = pipeline(model="text-generation", model_card="gpt2")

# # Instruções de início do chat
# print("Olá! Sou um assistente virtual. Pergunte-me sobre o clima ou digite 'sair' para encerrar o chat.")

# while True:
#     # Obter a entrada do usuário
#     user_input = input("Você: ")

#     # Verificar se o usuário quer sair
#     if user_input.lower() == "sair":
#         print("Até logo!")
#         break

#     # Gerar uma resposta do modelo
#     response = generator(user_input, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text']

#     # Exibir a resposta do assistente virtual
#     print("Assistente: ", response)

In [ ]:
# import os
# import io
# import IPython.display
# from PIL import Image
# import base64 
# import requests 
# requests.adapters.DEFAULT_TIMEOUT = 60

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file
# hf_api_key = os.environ['HF_API_KEY']

In [ ]:
# # Helper function
# import requests, json
# from text_generation import Client

# #FalcomLM-instruct endpoint on the text_generation library
# client = Client(os.environ['HF_API_FALCOM_BASE'], headers={"Authorization": f"Basic {hf_api_key}"}, timeout=120)

In [ ]:
# prompt = "Has math been invented or discovered?"
# client.generate(prompt, max_new_tokens=256).generated_text

In [ ]:
# #Back to Lesson 2, time flies!
# import gradio as gr
# def generate(input, slider):
#     output = client.generate(input, max_new_tokens=slider).generated_text
#     return output

# demo = gr.Interface(fn=generate, inputs=[gr.Textbox(label="Prompt"), gr.Slider(label="Max new tokens", value=20,  maximum=1024, minimum=1)], outputs=[gr.Textbox(label="Completion")])
# gr.close_all()
# demo.launch(share=True, server_port=int(os.environ['PORT1']))

In [ ]:
# import random

# def respond(message, chat_history):
#         #No LLM here, just respond with a random pre-made message
#         bot_message = random.choice(["Tell me more about it", 
#                                      "Cool, but I'm not interested", 
#                                      "Hmmmm, ok then"]) 
#         chat_history.append((message, bot_message))
#         return "", chat_history

# with gr.Blocks() as demo:
#     chatbot = gr.Chatbot(height=240) #just to fit the notebook
#     msg = gr.Textbox(label="Prompt")
#     btn = gr.Button("Submit")
#     clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

#     btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
#     msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit
# gr.close_all()
# demo.launch(share=True, server_port=int(os.environ['PORT2']))

In [ ]:
### adicionando contexto (histórico)

# def format_chat_prompt(message, chat_history):
#     prompt = ""
#     for turn in chat_history:
#         user_message, bot_message = turn
#         prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
#     prompt = f"{prompt}\nUser: {message}\nAssistant:"
#     return prompt

# def respond(message, chat_history):
#         formatted_prompt = format_chat_prompt(message, chat_history)
#         bot_message = client.generate(formatted_prompt,
#                                      max_new_tokens=1024,
#                                      stop_sequences=["\nUser:", "<|endoftext|>"] # retornando somente a resposta atual
#                                      ).generated_text
#         chat_history.append((message, bot_message))
#         return "", chat_history

# with gr.Blocks() as demo:
#     chatbot = gr.Chatbot(height=240) #just to fit the notebook
#     msg = gr.Textbox(label="Prompt")
#     btn = gr.Button("Submit")
#     clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

#     btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
#     msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit
# gr.close_all()
# demo.launch(share=True, server_port=int(os.environ['PORT3']))

In [ ]:
# def format_chat_prompt(message, chat_history, instruction):
#     prompt = f"System:{instruction}"
#     for turn in chat_history:
#         user_message, bot_message = turn
#         prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
#     prompt = f"{prompt}\nUser: {message}\nAssistant:"
#     return prompt

# def respond(message, chat_history, instruction, temperature=0.7):
#     prompt = format_chat_prompt(message, chat_history, instruction)
#     chat_history = chat_history + [[message, ""]]
#     stream = client.generate_stream(prompt,
#                                       max_new_tokens=1024,
#                                       stop_sequences=["\nUser:", "<|endoftext|>"],
#                                       temperature=temperature)
#                                       #stop_sequences to not generate the user answer
#     acc_text = ""
#     #Streaming the tokens
#     for idx, response in enumerate(stream):
#             text_token = response.token.text

#             if response.details:
#                 return

#             if idx == 0 and text_token.startswith(" "):
#                 text_token = text_token[1:]

#             acc_text += text_token
#             last_turn = list(chat_history.pop(-1))
#             last_turn[-1] += acc_text
#             chat_history = chat_history + [last_turn]
#             yield "", chat_history
#             acc_text = ""

# with gr.Blocks() as demo:
#     chatbot = gr.Chatbot(height=240) #just to fit the notebook
#     msg = gr.Textbox(label="Prompt")
#     with gr.Accordion(label="Advanced options",open=False):
#         system = gr.Textbox(label="System message", lines=2, value="A conversation between a user and an LLM-based AI assistant. The assistant gives helpful and honest answers.")
#         temperature = gr.Slider(label="temperature", minimum=0.1, maximum=1, value=0.7, step=0.1)
#     btn = gr.Button("Submit")
#     clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

#     btn.click(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot])
#     msg.submit(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot]) #Press enter to submit
# gr.close_all()
# demo.queue().launch(share=True, server_port=int(os.environ['PORT4']))

In [ ]:
# gr.close_all()